# 07-06: Anthropic Claude API使用 Anthropic SDK 调用 Claude 模型。

In [ ]:
// 安装: npm install @anthropic-ai/sdk
import Anthropic from '@anthropic-ai/sdk';
const anthropic = new Anthropic({
  apiKey: process.env.ANTHROPIC_API_KEY,
});
// ========== 1. 基础文本生成 ==========
async function basicCompletion() {
  const message = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    messages: [
      {
        role: 'user',
        content: 'What is the capital of France?'
      }
    ]
  });
  
  console.log(message.content[0].text);
  return message;
}

In [ ]:
// ========== 2. 多轮对话 ==========
async function conversation() {
  const messages = [
    {
      role: 'user' as const,
      content: 'Hello, my name is Alice.'
    }
  ];
  
  const response1 = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    messages
  });
  
  console.log('Assistant:', response1.content[0].text);
  
  // 继续对话
  messages.push(
    { role: 'assistant' as const, content: response1.content[0].text },
    { role: 'user' as const, content: 'What is my name?' }
  );
  
  const response2 = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    messages
  });
  
  console.log('Assistant:', response2.content[0].text);
}

In [ ]:
// ========== 3. 系统提示词 ==========
async function withSystemPrompt() {
  const message = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    system: 'You are a helpful coding assistant. Always provide code examples in TypeScript.',
    messages: [
      {
        role: 'user',
        content: 'How do I read a file in Node.js?'
      }
    ]
  });
  
  console.log(message.content[0].text);
}

In [ ]:
// ========== 4. 流式响应 ==========
async function streamingCompletion() {
  const stream = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    messages: [
      {
        role: 'user',
        content: 'Write a short story about AI'
      }
    ],
    stream: true
  });
  
  for await (const event of stream) {
    if (event.type === 'content_block_delta') {
      process.stdout.write(event.delta.text);
    }
  }
}

In [ ]:
// ========== 5. 工具调用 ==========
const tools = [
  {
    name: 'get_weather',
    description: 'Get the current weather in a given location',
    input_schema: {
      type: 'object',
      properties: {
        location: {
          type: 'string',
          description: 'The city and state, e.g. San Francisco, CA'
        },
        unit: {
          type: 'string',
          enum: ['celsius', 'fahrenheit'],
          description: 'The unit of temperature'
        }
      },
      required: ['location']
    }
  }
] as const;
async function toolUse() {
  const message = await anthropic.messages.create({
    model: 'claude-3-opus-20240229',
    max_tokens: 1024,
    tools,
    messages: [
      {
        role: 'user',
        content: 'What is the weather in San Francisco?'
      }
    ]
  });
  
  console.log('Stop reason:', message.stop_reason);
  
  if (message.stop_reason === 'tool_use') {
    const toolUse = message.content.find(c => c.type === 'tool_use');
    if (toolUse) {
      console.log('Tool use:', toolUse.name, toolUse.input);
      
      // 执行工具
      const weatherResult = await getWeather(
        (toolUse.input as any).location,
        (toolUse.input as any).unit
      );
      
      // 返回工具结果
      const finalMessage = await anthropic.messages.create({
        model: 'claude-3-opus-20240229',
        max_tokens: 1024,
        tools,
        messages: [
          {
            role: 'user',
            content: 'What is the weather in San Francisco?'
          },
          {
            role: 'assistant',
            content: message.content
          },
          {
            role: 'user',
            content: [
              {
                type: 'tool_result',
                tool_use_id: toolUse.id,
                content: JSON.stringify(weatherResult)
              }
            ]
          }
        ]
      });
      
      console.log('Final response:', finalMessage.content[0].text);
    }
  }
}
async function getWeather(location: string, unit?: string) {
  // Mock implementation
  return {
    location,
    temperature: unit === 'celsius' ? 20 : 68,
    condition: 'sunny'
  };
}

In [ ]:
// ========== 6. Vision - 图像理解 ==========
import * as fs from 'fs';
async function visionExample() {
  // 读取图片并转为 base64
  // const imageBuffer = fs.readFileSync('image.png');
  // const base64Image = imageBuffer.toString('base64');
  
  const message = await anthropic.messages.create({
    model: 'claude-3-opus-20240229',
    max_tokens: 1024,
    messages: [
      {
        role: 'user',
        content: [
          {
            type: 'text',
            text: 'What do you see in this image?'
          },
          {
            type: 'image',
            source: {
              type: 'base64',
              media_type: 'image/png',
              data: 'base64_encoded_image_data_here'
            }
          }
        ]
      }
    ]
  });
  
  console.log(message.content[0].text);
}

In [ ]:
// ========== 7. JSON Mode ==========
async function jsonMode() {
  const message = await anthropic.messages.create({
    model: 'claude-3-sonnet-20240229',
    max_tokens: 1024,
    messages: [
      {
        role: 'user',
        content: `Extract the name, age, and city from this text and return as JSON:
        John is 25 years old and lives in New York.`
      }
    ]
  });
  
  // Claude 不直接支持 JSON mode，但可以要求输出 JSON
  try {
    const json = JSON.parse(message.content[0].text);
    console.log(json);
  } catch {
    console.log('Raw output:', message.content[0].text);
  }
}